# Рекомендация тарифов

## Описание

Имеются данные о поведении клиентов, использующих некоторые тарифы компании. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

In [1]:
import pandas
import pandas as pd
import sklearn
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
import warnings

Импортировал необходимые для работы библиотеки.

In [2]:
data = pandas.read_csv(f'/datasets/users_behavior.csv')
data

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
data.value_counts()

calls  minutes  messages  mb_used   is_ultra
0.0    0.00     0.0       530.78    1           1
75.0   529.66   20.0      21749.32  0           1
       487.72   60.0      25759.17  0           1
       489.47   7.0       20478.94  1           1
       496.06   14.0      13850.84  1           1
                                               ..
48.0   334.49   19.0      17538.01  0           1
       336.38   14.0      21902.41  0           1
       338.06   3.0       20883.01  0           1
       341.65   0.0       19000.02  0           1
244.0  1632.06  39.0      9756.91   1           1
Length: 3214, dtype: int64

In [6]:
data['calls'] = data['calls'].astype('int')
data['messages'] = data['messages'].astype('int')

In [7]:
spearmanr(data['calls'], data['messages'])

SpearmanrResult(correlation=0.15503223116820483, pvalue=9.642126018301472e-19)

Можем заметить, что корелляция между количеством звонков и сообщений очень слабая.

### Вывод: 
Можно заметить, что пропусков в данных нет и форматы данных везде подходящие, разве что - заменил типы данных в колонках messages и calls на int, т.к. количество сообщений и звонков - целочисленные значения.

## Разделение данных на выборки

In [8]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [10]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, random_state=12345)
print('Размер выборки для тренировочных данных:', len(features_train))
print('Размер выборки для тестовых данных:', len(features_test))
print('Размер выборки для валидационных данных:', len(features_valid))

Размер выборки для тренировочных данных: 1928
Размер выборки для тестовых данных: 643
Размер выборки для валидационных данных: 643


Создал три выборки:
 - train - для тренировочных данных
 - valid - для валидационных данных
 - test - для тестовых данных

## Исследуйте модели

### Проверим работу Decision Tree

In [11]:
best_model = None
best_acc_dt = 0
best_depth = 0
for depth in range(2,50):
    dt_model = DecisionTreeClassifier(random_state=50, max_depth=depth)
    dt_model.fit(features_train, target_train)
    prediction_dt = dt_model.predict(features_valid)
    acc = accuracy_score(prediction_dt, target_valid)
    if best_acc_dt < acc:
        best_model = dt_model
        best_acc_dt = acc
        best_depth = depth

print(best_depth, best_acc_dt)

9 0.7869362363919129


Вывод: Точность прогноза модели Decision Tree равна 0.7869362363919129 (При глубине равной девяти)

### Проверим работу Logistic Regression

In [12]:
warnings.filterwarnings('ignore')
best_log_reg = None
best_res = 0
solver = ['newton-cg', 'sag', 'saga', 'liblinear', 'lbfgs']

for i in solver:
    log_reg = LogisticRegression(random_state=50, solver=i, max_iter=10000)
    log_reg.fit(features_train, target_train)
    log_reg_prediction = log_reg.predict(features_valid)
    log_reg_acc = accuracy_score(log_reg_prediction, target_valid)
    if log_reg_acc > best_res:
        best_res = log_reg_acc
        best_log_reg = log_reg
        best_solver = i

print(best_solver, log_reg_acc)

newton-cg 0.7107309486780715


Вывод: Лучшая точность прогноза модели Logistic Regression (достигается при использовании решателя newton-cg) равна newton-cg 0.7107309486780715.

### Проверим работу Random Forest

In [13]:
#from tqdm import tqdm

best_rf = None
best_acc_rf = 0
best_est_rf = 0
best_depth_rf = 0
for f in range(2, 5):
    for est in range(10, 200, 10):
        for depth in range(2,25):
            rf = RandomForestClassifier(random_state=50, n_estimators=est, max_depth=depth, max_features=f)
            rf.fit(features_train, target_train)
            prediction_rf = rf.predict(features_valid)
            acc_rf = accuracy_score(prediction_rf, target_valid)
            if best_acc_rf < acc_rf:
                best_rf = rf
                best_depth_rf = depth
                best_est_rf = est
                best_acc_rf = acc_rf
                best_f_rf = f
                
print(best_depth_rf, best_est_rf, best_f_rf, best_acc_rf)

7 150 2 0.8055987558320373


Вывод: Лучшая точность прогноза модели Random Forest равна 0.8055987558320373 (При глубине равной семи, 150 оценщиках (деревьях) и с двумя признаками).

### Вывод 

Самый точный прогноз дает модель Random Forest. Точность прогноза 0.8055987558320373.

## Проверка лучшей модели на тестовой выборке

Проверка случайного леса на тестовой выборке:

In [14]:
pred_testsample_rf = best_rf.predict(features_test)
acc_testsample_rf = accuracy_score(pred_testsample_rf, target_test)
print(acc_testsample_rf)

0.7993779160186625


### Вывод

Как и в случае с тренировочной выборкой, доля правильных ответов у модели типа "Случайный лес" высока и выдает достаточно хороший результат 0.7993779160186625 или 79,9% правильных ответов.

## Проверка модели на адекватность

Используем классификатор для сравнения нашей модели с его прогнозом.

In [15]:
dummy_1 = DummyClassifier(strategy="most_frequent", random_state=0).fit(features_train, target_train)
dummy_prediction = dummy_1.predict(features_test)
dummy_acc = accuracy_score(target_test, dummy_prediction)
print("Точность дамми-модели:", dummy_acc)

Точность дамми-модели: 0.6842923794712286


### Вывод: 
Заметно, что такая модель дает правильные ответы значительно (на 11,5%) реже модели случайного леса, примененной ранее. Следовательно, наша модель проходит проверку на адекватность.

# Общий вывод:
Наиболее точной моделью нашего исследования является "Случайный лес".